##### Load packages

In [ ]:


GPy.__version__

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import time

# Set pandas view options
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# filter warnings messages from the notebook
import warnings
warnings.filterwarnings('ignore')

from matminer.datasets.dataset_retrieval import load_dataset
## Generate magpie features
from matminer.featurizers.composition import ElementProperty
from pymatgen import Composition

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, train_test_split, learning_curve, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn import metrics
from sklearn import preprocessing 
import sklearn.gaussian_process as gp
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

from pymatgen import MPRester, Composition
mpr = MPRester('68rWEneaZFyIaKh15uKr') # provide your API key here or add it to pymatgen

# Load & Process Data

In [ ]:
featurized_data = pd.read_csv('featurized_brgoch_data.csv', index_col=0).drop(columns=['index']).dropna()
featurized_data.head(4)

In [ ]:
exp_data = featurized_data.loc[featurized_data['expt_calculated']==1].reset_index(drop=True).drop(columns=['expt_calculated'])
theory_data = featurized_data.loc[featurized_data['expt_calculated']==0].reset_index(drop=True).drop(columns=['expt_calculated'])

In [ ]:
print(len(exp_data), len(theory_data))

In [ ]:
def get_features_from_df(df):
    features_df = df.drop(columns=['reduced_formula', 'bandgap', 'composition', 'index']).copy()
    return features_df

X = get_features_from_df(featurized_data).values.tolist()                              
y = np.array(featurized_data[['bandgap']])

In [ ]:
x_scaler = preprocessing.StandardScaler()
pca = PCA(n_components=4)
preprocessing_pipeline = Pipeline([('scaler', x_scaler), ('pca', pca)])

scaled_y = preprocessing.scale(y)
X_train, X_test, y_train, y_test = train_test_split(X, scaled_y, test_size=0.8, random_state=42)

processed_X_train = pca.fit_transform(X_train)
print(np.sum(pca.explained_variance_ratio_))

# Gaussian Process 

### Model Selection 
The form of the mean function and covariance kernel function in the GP prior is chosen and tuned during model selection. The mean function is typically constant, either zero or the mean of the training dataset. There are many options for the covariance kernel function: it can have many forms as long as it follows the properties of a kernel (i.e. semi-positive definite and symmetric). Some common kernel functions include constant, linear, square exponential and Matern kernel, as well as a composition of multiple kernels.
A popular kernel is the composition of the constant kernel with the radial basis function (RBF) kernel, which encodes for smoothness of functions (i.e. similarity of inputs in space corresponds to the similarity of outputs)

This kernel has two hyperparameters: signal variance, σ², and lengthscale, l. In scikit-learn, we can chose from a variety of kernels and specify the initial value and bounds on their hyperparameters.

In [ ]:
kernel = gp.kernels.ConstantKernel(1.0, (1e-1, 1e3)) * gp.kernels.RBF(10.0, (1e-3, 1e3))

After specifying the kernel function, we can now specify other choices for the GP model in scikit-learn. For example, alpha is the variance of the i.i.d. noise on the labels, and normalize_y refers to the constant mean function — either zero if False or the training data mean if True.

In [ ]:
model = gp.GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=True)

A popular approach to tune the hyperparameters of the covariance kernel function is to maximize the log marginal likelihood of the training data. A gradient-based optimizer is typically used for efficiency; if unspecified above, the default optimizer is ‘fmin_l_bfgs_b’. Because the log marginal likelihood is not necessarily convex, multiple restarts of the optimizer with different initializations is used (n_restarts_optimizer).

In [ ]:
%%time
model.fit(processed_X_train, y_train)
params = model.kernel_.get_params()

In [ ]:
params 

To calculate the predictive posterior distribution, the data and the test observation is conditioned out of the posterior distribution. Again, because we chose a Gaussian process prior, calculating the predictive distribution is tractable, and leads to normal distribution that can be completely described by the mean and covariance. 

In [ ]:
processed_X_test = pca.transform(X_test)
y_pred, std = model.predict(processed_X_test, return_std=True)

To measure the performance of the regression model on the test observations, we can calculate the mean squared error (MSE) on the predictions.

In [ ]:
mae = metrics.mean_absolute_error(y_pred, y_test)

In [ ]:
mae

In [ ]:
plt.figure(figsize=(15,10))
plt.errorbar(y_test, y_pred, yerr=std, linestyle="None", marker='o', mec='blue',
             ecolor='yellowgreen')
plt.xlabel('Calculated')
plt.ylabel('Predicted')
# plt.ylim(-50, 50)
plt.show()